In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd  drive/MyDrive/Colab Notebooks/'Deepfake Asnmnt'/

/content/drive/MyDrive/Colab Notebooks/Deepfake Asnmnt


In [3]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00


In [4]:
import timm 
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from PIL import Image
from sklearn.metrics import precision_recall_fscore_support

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
# num_classes = 2
# xc_model = timm.create_model('xception', pretrained = True, num_classes = num_classes).to(device)
# ef_model = timm.create_model('efficientnet_b0', pretrained=True, num_classes = num_classes).to(device)

# criterion = nn.BCEWithLogitsLoss()
# optimizer_xc = optim.Adam(xc_model.parameters(), lr=0.001)
# optimizer_ef = optim.Adam(ef_model.parameters(), lr=0.001)

# x     = torch.randn(32, 3, 224, 224).to(device) # 32 is batch_size
# ef_model(x).shape

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


In [7]:
class BaseDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        self.imgs = []
        self.labels = []
        
        for file in os.listdir(data_path):
            if file.endswith('.png'):
                img_path = os.path.join(data_path, file)
                label = file.split('_')[0]  # 파일 이름에서 레이블 파싱
                self.imgs.append(img_path)
                self.labels.append(label)
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        img = Image.open(self.imgs[idx])
        img = self.transforms(img)
        label = 1 if self.labels[idx] == 'fake' else 0
        return img, label

In [8]:
# Train models
class Experiment():
    def __init__(self):
        self.num_classes = 2
        self.xc_model = timm.create_model('xception', pretrained = True, num_classes = self.num_classes).to(device)
        self.ef_model = timm.create_model('efficientnet_b0', pretrained=True, num_classes = self.num_classes).to(device)

        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer_xc = optim.Adam(self.xc_model.parameters(), lr=0.001)
        self.optimizer_ef = optim.Adam(self.ef_model.parameters(), lr=0.001)


    def train_model(self, trainloader, valloader):
        num_epochs = 100

        xc_train_accs = []
        xc_val_accs = []
        ef_train_accs = []
        ef_val_accs = []

        for epoch in range(num_epochs):
            # Training loop for Xception model
            self.xc_model.train()
            xc_total_loss = 0
            xc_total_correct = 0
            for images, labels in trainloader:
                images = images.to(device)
                labels = labels.to(device)
                self.optimizer_xc.zero_grad()
                outputs = self.xc_model(images)
                labels_onehot = nn.functional.one_hot(labels, num_classes = self.num_classes).float()
                loss = self.criterion(outputs, labels_onehot)
                loss.backward()
                self.optimizer_xc.step()

                xc_total_loss += loss.item()
                xc_total_correct += (outputs.argmax(dim=1) == labels).sum().item()

            xc_train_loss = xc_total_loss / len(trainloader)
            xc_train_acc = xc_total_correct / len(trainloader.dataset)
            xc_train_accs.append(xc_train_acc)

            print(f'Epoch {epoch+1} - Xception model - Training loss: {xc_train_loss:.4f} - Training accuracy: {xc_train_acc:.4f}')

            # Training loop for EfficientNet model
            self.ef_model.train()
            ef_total_loss = 0
            ef_total_correct = 0
            for images, labels in trainloader:
                images = images.to(device)
                labels = labels.to(device)
                self.optimizer_ef.zero_grad()
                outputs = self.ef_model(images)
                labels_onehot = nn.functional.one_hot(labels, num_classes=self.num_classes).float()
                loss = self.criterion(outputs, labels_onehot)
                loss.backward()
                self.optimizer_ef.step()

                ef_total_loss += loss.item()
                ef_total_correct += (outputs.argmax(dim=1) == labels).sum().item()

            ef_train_loss = ef_total_loss / len(trainloader)
            ef_train_acc = ef_total_correct / len(trainloader.dataset)
            ef_train_accs.append(ef_train_acc)

            print(f'Epoch {epoch+1} - EfficientNet model - Training loss: {ef_train_loss:.4f} - Training accuracy: {ef_train_acc:.4f}')


            # Validation loop
            self.xc_model.eval()
            self.ef_model.eval()
            with torch.no_grad():
                xc_total, xc_correct = 0, 0
                for images, labels in valloader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = self.xc_model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    xc_total += labels.size(0)
                    xc_correct += (predicted == labels).sum().item()
                xc_accuracy = xc_correct / xc_total
                xc_val_accs.append(xc_accuracy)
            
                print(f'Epoch {epoch+1} - Xception model - Validation accuracy: {xc_accuracy:.4f}')

                ef_total, ef_correct = 0, 0
                for images, labels in valloader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = self.ef_model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    ef_total += labels.size(0)
                    ef_correct += (predicted == labels).sum().item()
                ef_accuracy = ef_correct / ef_total
                ef_val_accs.append(ef_accuracy)

                print(f'Epoch {epoch+1} - EfficientNet model - Validation accuracy: {ef_accuracy:.4f}')

        plt.plot(xc_train_accs, label='Xception Training Accuracy')
        plt.plot(xc_val_accs, label='Xception Validation Accuracy')
        plt.plot(ef_train_accs, label='EfficientNet Training Accuracy')
        plt.plot(ef_val_accs, label='EfficientNet Validation Accuracy')
        plt.legend()
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.show()

    def test_model(self, testloader):

        self.xc_model.eval()
        self.ef_model.eval()
        with torch.no_grad():
            xc_total, xc_correct = 0, 0
            xc_true_labels, xc_predicted_labels = [], []
            for images, labels in testloader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = self.xc_model(images)
                _, predicted = torch.max(outputs.data, 1)
                xc_total += labels.size(0)
                xc_correct += (predicted == labels).sum().item()
                xc_true_labels += labels.cpu().tolist()
                xc_predicted_labels += predicted.cpu().tolist()
            xc_accuracy = xc_correct / xc_total
            xc_precision, xc_recall, xc_f1, _ = precision_recall_fscore_support(xc_true_labels, xc_predicted_labels, average='binary')
        
            print(f'Xception model - Test accuracy: {xc_accuracy:.4f}')
            print(f'Xception model - Test precision: {xc_precision:.4f}')
            print(f'Xception model - Test recall: {xc_recall:.4f}')
            print(f'Xception model - Test F1 score: {xc_f1:.4f}')


            ef_total, ef_correct = 0, 0
            ef_true_labels, ef_predicted_labels = [], []
            for images, labels in testloader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = self.ef_model(images)
                _, predicted = torch.max(outputs.data, 1)
                ef_total += labels.size(0)
                ef_correct += (predicted == labels).sum().item()
                ef_true_labels += labels.cpu().tolist()
                ef_predicted_labels += predicted.cpu().tolist()
            ef_accuracy = ef_correct / ef_total
            ef_precision, ef_recall, ef_f1, _ = precision_recall_fscore_support(ef_true_labels, ef_predicted_labels, average='binary')

            print(f'EfficientNet model - Test accuracy: {ef_accuracy:.4f}')
            print(f'EfficientNet model - Test precision: {ef_precision:.4f}')
            print(f'EfficientNet model - Test recall: {ef_recall:.4f}')
            print(f'EfficientNet model - Test F1 score: {ef_f1:.4f}')

In [10]:
#1) HighQuality-Face2Face

hf_train_dataset = BaseDataset(data_path = "High Quality/f2f_data/train")
hf_trainloader = DataLoader(hf_train_dataset, batch_size=32, shuffle=True)
hf_val_dataset = BaseDataset(data_path = "High Quality/f2f_data/val")
hf_valloader = DataLoader(hf_val_dataset, batch_size=32, shuffle=True)
hf_test_dataset = BaseDataset(data_path = "High Quality/f2f_data/test")
hf_testloader = DataLoader(hf_test_dataset, batch_size=32, shuffle=True)

hf = Experiment()
hf.train_model(hf_trainloader, hf_valloader)
hf.test_model(hf_testloader)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Xception model - Test accuracy: 0.5000
Xception model - Test precision: 0.2500
Xception model - Test recall: 0.5000
Xception model - Test F1 score: 0.3333
EfficientNet model - Test accuracy: 0.5025
EfficientNet model - Test precision: 0.5015
EfficientNet model - Test recall: 0.8141
EfficientNet model - Test F1 score: 0.6207


In [11]:
#2) HighQuality-NeuralTexture

hn_train_dataset = BaseDataset(data_path = "High Quality/nt_data/train")
hn_trainloader = DataLoader(hn_train_dataset, batch_size=32, shuffle=True)
hn_val_dataset = BaseDataset(data_path = "High Quality/nt_data/val")
hn_valloader = DataLoader(hn_val_dataset, batch_size=32, shuffle=True)
hn_test_dataset = BaseDataset(data_path = "High Quality/nt_data/test")
hn_testloader = DataLoader(hn_test_dataset, batch_size=32, shuffle=True)

hn = Experiment()
hn.train_model(hn_trainloader, hn_valloader)
hn.test_model(hn_testloader)

KeyboardInterrupt: ignored

In [ ]:
#3) LowQuality-Face2Face

lf_train_dataset = BaseDataset(data_path = "Low Quality/f2f_data/train")
lf_trainloader = DataLoader(lf_train_dataset, batch_size=32, shuffle=True)
lf_val_dataset = BaseDataset(data_path = "Low Quality/f2f_data/val")
lf_valloader = DataLoader(lf_val_dataset, batch_size=32, shuffle=True)
lf_test_dataset = BaseDataset(data_path = "Low Quality/f2f_data/test")
lf_testloader = DataLoader(lf_test_dataset, batch_size=32, shuffle=True)

lf = Experiment()
lf.train_model(lf_trainloader, lf_valloader)
lf.test_model(lf_testloader)

In [ ]:
#4) LowQuality-NeuralTexture

ln_train_dataset = BaseDataset(data_path = "Low Quality/nt_data/train")
ln_trainloader = DataLoader(ln_train_dataset, batch_size=32, shuffle=True)
ln_val_dataset = BaseDataset(data_path = "Low Quality/nt_data/val")
ln_valloader = DataLoader(ln_val_dataset, batch_size=32, shuffle=True)
ln_test_dataset = BaseDataset(data_path = "Low Quality/nt_data/test")
ln_testloader = DataLoader(ln_test_dataset, batch_size=32, shuffle=True)

ln = Experiment()
ln.train_model(ln_trainloader, ln_valloader)
ln.test_model(ln_testloader)